Combined all building datas and create new Builiding column

# Import

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
# parameters search
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from bayes_opt import BayesianOptimization

# models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from xgboost import XGBRegressor
import catboost as cb
import lightgbm as lgb
# To: install catboost
# !pip3 install catboost

from datetime import datetime
import os
import pathlib
import matplotlib.pyplot as plt

# 1. Load Data

In [ ]:
# Create a list to add csv files as df
Bldg = []

# Read all building data and append to list
for path in pathlib.Path("../Data/microclimate_model/Combined/dataset1").iterdir():
        if path.is_file():
            current_file = pd.read_csv(path)
            current_file = current_file.drop(columns=['Unnamed: 0'])
            Bldg.append(current_file)
    

## 1.1 Preprocessing 

1. Adding Month, Hour, and Minute to data
2. Removing hours out of ENVI-met accuracy range (after 9 pm)
3. Add CHWTON/SQFT to columns using condition area for each building taken from
    https://fdm-apps.asu.edu/UFRM/FDS/FacilityData.aspx
4. Drop na rows in limited data (some data points from campus metabolism not available)
5. Add Absolute Humidity to Data


In [ ]:
# Create Month, Hour, and Minute column for all dataframes in list
for i in range(len(Bldg)):
    Bldg[i].Date_Time = pd.to_datetime(Bldg[i].Date_Time)
    Bldg[i]['Month_num'] = Bldg[i].Date_Time.dt.month
    Bldg[i]['Hour_num'] = Bldg[i].Date_Time.dt.hour
    Bldg[i]['Minute_num'] = Bldg[i].Date_Time.dt.minute
    Bldg[i]['Day_num'] = Bldg[i].Date_Time.dt.day

# Remove data after 9pm
for i in range(len(Bldg)):
    Bldg[i] = Bldg[i][(Bldg[i]['Hour_num'] <= 20) & (Bldg[i]['Hour_num'] > 0)]

# Add Column: CHWTON/Condition Area (SqFt) or ['CHWTON/SQFT']
cond_area = {'Noble Library':88658,'Biodesign B':132215,'Biodesign C':145410,
             'Biodesign A':133016,'Psychology':69864,'Goldwater':165237,'Schwada COB':99857,
             'ISTB 2':41404,'Bulldog Hall':68067,'ISTB 4':231646,'Psychology North':43034}
for i in range(len(Bldg)):
    if Bldg[i]['bldgname'][0] in cond_area:
        Bldg[i]['CHWTON/SQFT'] = Bldg[i]['CHWTON'] / cond_area[Bldg[i]['bldgname'][0]]



Absolute Humidity Equations: https://www.hatchability.com/Vaisala.pdf. To test, compare with https://planetcalc.com/2167/ but notice the calculation made here is in g/m^3 and in this website it is in Kg/m^3.


In [ ]:
# Check if NA in data
for i in range(len(Bldg)):
    null_data = Bldg[i][Bldg[i].isnull().any(axis=1)]
#     print(null_data)

In [ ]:
# Drop NA rows in data
for i in range(len(Bldg)):
    Bldg[i] = Bldg[i].dropna()
    Bldg[i] = Bldg[i].reset_index(drop=True)

In [ ]:
# Convert Rel Hum to Abs Hum
for i in range(len(Bldg)):
    T_i = Bldg[i]['Air Temp']
    RH = Bldg[i]['Rel Hum']/100

    T = T_i + 273.15
    P_c = 220640
    T_c = 647.096
    C_1 = -7.85951783
    C_2 = 1.84408259
    C_3 =  -11.7866497
    C_4 = 22.6807411
    C_5 = -15.9618719
    C_6 = 1.80122502
    v = 1 - (T/T_c)

    x = (T_c/T)*((C_1*v) + (C_2*np.power(v, 1.5)) + (C_3*np.power(v, 3)) 
                 + (C_4*np.power(v, 3.5)) + (C_5*np.power(v, 4)) + (C_6*np.power(v, 7.5))) 

    P_ws = np.exp(x)*P_c
    P_w = P_ws*RH

    C = 2.16679
    A = C*P_w*100/T

    Bldg[i]['Abs Hum'] = A

In [ ]:
Bldg[0].Date.unique()

Month available:<br>
May: 16, 23 <br>
June: 7, 8, 20, 21, 25, 26<br>
August: 3, 27<br>
September: 11, 29<br>

### 2.2.3 Correlation Matrix

In [ ]:
corr_pd = pd.DataFrame(Bldg[bldgnum][['Time','Air Temp','Rel Hum','KW','HTmmBTU','Month_num','Hour_num','Day_num', 'CHWTON/SQFT', 'Abs Hum']])
corrMatrix = corr_pd.corr()
sns.heatmap(corrMatrix)


# 3. All Buildings

In [ ]:
Bldg_df = pd.DataFrame()

In [ ]:
for i in range(len(Bldg)):
    Bldg_df = Bldg_df.append(Bldg[i])

In [ ]:
Bldg_df.reset_index(drop = True , inplace = True)

In [ ]:
Bldg_df

## 3.1 EDA

### 3.1.1 Boxplots

In [ ]:
# Create List of building names so we can extract the name easily 
BldgName = ["Noble Library","Biodesign B","Biodesign C",
              "Biodesign A", "Psychology", "Goldwater",
              "Schwada COB", "ISTB 2", "Bulldog Hall",
              "ISTB 4", "Pyschology North"]


In [ ]:
#Create CHWTON boxplots for all buildings #
def createBoxPlot(df, columnName, BldgName):
    row_size = 6
    column_size = 2
    fig, ax = plt.subplots(row_size, column_size, figsize = (15,40))

    i = 0
    while i < (len(df)):
        for row in range(row_size):
            for col in range(column_size):
                if i < len(df):
                    df[i].boxplot(by='Hour_num',
                                    column=[columnName],
                                    grid = False,
                                    figsize = (5,5),
                                    ax = ax[row,col] )
                    ax[row,col].title.set_text(BldgName[i])
                    i += 1

    fig.suptitle(columnName + ' Boxplot by Hour')
    plt.show()
    
createBoxPlot(Bldg, 'CHWTON', BldgName)

In [ ]:
createBoxPlot(Bldg, 'Air Temp', BldgName)

### 3.1.2 Time Series

In [ ]:
## Print CHWTON/SQFT for all buildings and all timestamps in data
ax = Bldg[0]['CHWTON/SQFT'].plot(figsize = (15,9))
legendlabels = []
for i in range(len(Bldg)-1):
    Bldg[i+1]['CHWTON/SQFT'].plot(ax=ax)
    legendlabels.append(Bldg[i].bldgname[0])
    
ax.legend(labels = legendlabels)


## 3.2 Feature Engineering

In [ ]:
# function to encode df columns into sine and cosine
def encode(df, col, max_val):
    df[col.replace('_num', '') + '_sin'] = np.sin(2 * np.pi * df[col]/max_val)
    df[col.replace('_num', '') + '_cos'] = np.cos(2 * np.pi * df[col]/max_val)
    df.drop(columns = [col], inplace = True)
    return df


In [ ]:
# create a list of df for buildings with cyclical time features
Bldg_cyclic = []

Bldg_enc = Bldg_df.copy(deep = True)
Bldg_enc = encode(Bldg_enc, 'Minute_num', 60.0)
Bldg_enc = encode(Bldg_enc, 'Hour_num', 23.0)
Bldg_enc = encode(Bldg_enc, 'Day_num', 30.0)
Bldg_enc = encode(Bldg_enc, 'Month_num', 12.0)
Bldg_cyclic = Bldg_enc
    
# Plot cyclical features sample
fig, ax = plt.subplots(2,2, figsize = (8,7))
Bldg_cyclic.plot.scatter('Minute_sin', 'Minute_cos', ax = ax[0,0]).set_aspect('equal')
Bldg_cyclic.plot.scatter('Hour_sin', 'Hour_cos', ax = ax[0,1]).set_aspect('equal')
Bldg_cyclic.plot.scatter('Day_sin', 'Day_cos', ax = ax[1,0]).set_aspect('equal')
Bldg_cyclic.plot.scatter('Month_sin', 'Month_cos', ax = ax[1,1]).set_aspect('equal')

In [ ]:
Bldg_cyclic.reset_index(drop = True, inplace = True)

In [ ]:
Bldg_cyclic

## 3.3 Modelling set up

In [ ]:
### Create scores dataframe and add building names ###
scores_df = pd.DataFrame()


In [ ]:
# function to train a model and get its scores
def trainAndGetScore(pModel, pModelName, pDf_all_bldg, pDf_scores):
    # 1. drop na values if in dataframe
    if (pDf_all_bldg.isnull().values.any() == True):
        pDf_all_bldg = pDf_all_bldg.dropna()
            
    # 2. split data into X and y
    X = pDf_all_bldg.drop(columns=['Date', 'Time','bldgname','HTmmBTU', 'Date_Time', 'CHWTON', 'CHWTON/SQFT'])
    y = pDf_all_bldg['CHWTON']    
        
    # 3. Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)
        
    # 4. fit model that already has parameters
    pModel.fit(X_train, y_train)
        
    # 5. Get prediction
    y_pred = pModel.predict(X_test)
    ModelPred = pd.DataFrame({'Actual CHWTON':y_test, 'Predicted CHWTON':y_pred})
    ModelPred = ModelPred.sort_index()
    
    # 6. Get best params if it's a random or grid search
    if("random" in pModelName) or ("grid" in pModelName):
        print(pModel.best_estimator_.get_params())
        
    # Save scores
    score = pModel.score(X_test, y_test)
#     i = scores_df.shape[0] # get last index in the df
    pDf_scores[pModelName] = score

## 3.4 Model 1: Random Forest


### 3.4.1 No Tuning

In [ ]:
RF_base = RandomForestRegressor(n_estimators = 100, random_state = 42)

# 2. split data into X and y
X = Bldg_df.drop(columns=['Date', 'Time','bldgname','HTmmBTU', 'Date_Time', 'CHWTON', 'CHWTON/SQFT'])
y = Bldg_df['CHWTON'] 


# 3. Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)

RF_base.fit(X_train, y_train)

In [ ]:
y_pred = RF_base.predict(X_test)
y_pred
score = RF_base.score(X_test, y_test)

In [ ]:
RF_base = RandomForestRegressor(n_estimators = 100, random_state = 42)

# 1. Base RF on base data
trainAndGetScore(RF_base, "RF_base", Bldg_df, scores_df)

# 2. Base RF on cyclical time features
trainAndGetScore(RF_base, "RF_cyclic", Bldg_cyclic, scores_df)


In [ ]:
scores_df

### 3.4.2 Random Search Tuning

In [ ]:
# Define parameters for RF

# 1. Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 50, stop = 500, num = 10)]

# 2. Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# 3. Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# 4. Minimum number of samples required at each leaf node
min_samples_leaf = [ 1, 2, 4]

# 5. Method of selecting samples for training each tree
bootstrap = [True, False]

# 6. Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

print(random_grid)

In [ ]:
# 1. Set parameters on random_RF
RF_random = RandomizedSearchCV(estimator = RF_base,
                               param_distributions = random_grid,
                               n_iter = 20, cv = 5,
                               verbose = 2,
                               scoring ='r2',
                               random_state = 42,
                               n_jobs = -1)

# 2. Train on base data
trainAndGetScore(RF_random, "RF_random", Bldg_df, scores_df)

# 3. Train on data with cyclical time features
trainAndGetScore(RF_random, "RF_random_cyclic", Bldg_cyclic, scores_df)
scores_df

In [ ]:
print(RF_random.best_estimator_.get_params())

### 3.4.3 Grid Search Tuning

In [ ]:
param_grid = {'n_estimators': [200, 220, 230,240],
               'max_features': ["sqrt"],
               'max_depth': [17, 20, 22],
               'min_samples_split': [2,3,4],
               'min_samples_leaf': [ 1, 2],
               'bootstrap': bootstrap}

In [ ]:
# 1. Set parameters on random_RF
RF_grid = GridSearchCV(estimator = RF_base,
                       param_grid = param_grid,
                       cv = 5,
                       scoring ='r2',
                       n_jobs = -1)

# 2. Train on base data
trainAndGetScore(RF_grid, "RF_grid", Bldg_df, scores_df)

# 3. Train on data with cyclical time features
trainAndGetScore(RF_grid, "RF_grid_cyclic", Bldg_cyclic, scores_df)
scores_df

## 3.5 Model 2: XGBoost

### 3.5.1. No Tuning

In [ ]:
# 1. create base model
XGB_base = XGBRegressor(n_estimators = 100, random_state = 42)

# 2. Base XGB on base data
trainAndGetScore(XGB_base, "XGB_base", Bldg_df, scores_df)

# 3. Base XGB on cyclica time features
trainAndGetScore(XGB_base, "XGB_cyclic", Bldg_cyclic, scores_df)
scores_df

### 3.5.2. Random Search Tuning

In [ ]:
# 1. Define grid
params = {
    'n_estimators':[ 100, 250, 500, 1000],
    'min_child_weight':[4,5,8], 
    'gamma':[i/10.0 for i in range(3,6)],  
    'subsample':[i/10.0 for i in range(6,11)],
    'colsample_bytree':[i/10.0 for i in range(6,11)], 
    'max_depth': [2,3,4,6,7],
    'objective': ['reg:squarederror', 'reg:tweedie'],
    'booster': ['gbtree', 'gblinear'],
    'eval_metric': ['rmse'],
    'eta': [i/10.0 for i in range(3,6)],
}

# 2. Set up model with grid
n_iter_search = 20
XGB_random = RandomizedSearchCV(XGB_base,
                                param_distributions = params,
                                n_iter = n_iter_search,
                                cv = 5,
                                verbose = 2,
                                random_state = 42,
                                scoring ='r2',
                                n_jobs = -1)

# 2. Train on base data
trainAndGetScore(XGB_random, "XGB_random", Bldg_df, scores_df)

# 3. Train on data with cyclical time features
trainAndGetScore(XGB_random, "XGB_random_cyclic", Bldg_cyclic, scores_df)
scores_df

### 3.5.3 Bayesian Opt. tuning

In [ ]:
# from bayes_opt import BayesianOptimization


# for i in range(len(Bldg)):
#         # 1. drop na values if in dataframe
#         if (Bldg[i].isnull().values.any() == True):
#             Bldg[i] = pDf_all_bldg[i].dropna()
            
#         # 2. split data into X and y
#         X = Bldg[i].drop(columns=['Date', 'Time','bldgname','HTmmBTU', 'Date_Time', 'CHWTON', 'CHWTON/SQFT'])
#         y = Bldg[i]['CHWTON']    
        
#         # 3. Train-Test Split
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20)
    
# # 1. Function to maximimse score 
# # This function will take the the parameters we defined and obtain a score
# def train_XGB(max_depth, gamma, n_estimators, learning_rate, subsample):
#     params_XGB = {}
#     params_XGB['max_depth'] = round(max_depth)
#     params_XGB['gamma'] = gamma
#     params_XGB['learning_rate'] = learning_rate
#     params_XGB['n_estimators'] = round(n_estimators)
#     params_XGB['subsample'] = subsample
#     scores = cross_val_score(XGBRegressor(random_state=42, **params_XGB),
#                              X_train, y_train, scoring='r2', cv=5).mean()
#     score = scores.mean()
#     return score
               
               
               
# params_XGB ={
#     'max_depth':(3, 10),
#     'gamma':(0, 1),
#     'learning_rate':(0.01, 1),
#     'n_estimators':(80, 150),
#     'subsample': (0.8, 1)
# }
# XGB_bayes = BayesianOptimization(train_XGB, params_XGB, random_state=111)
# XGB_bayes.maximize(init_points=20, n_iter=10)
# print('It takes %s minutes' % ((time.time() - start)/60))

## 3.6 Model 3: LightGBM

### 3.6.1 No Tuning

In [ ]:
LGBM_base = lgb.LGBMRegressor(random_state = 42)

# 2. Base LGBM on base data
trainAndGetScore(LGBM_base, "LGBM_base", Bldg_df, scores_df)

# 3. Base LGBM on cyclica time features
trainAndGetScore(LGBM_base, "LGBM_cylic", Bldg_cyclic, scores_df)
scores_df

### 3.6.2 Random Search Tuning

In [ ]:
# 1. Define grid
random_grid = {
    'num_leaves': [7, 14, 21, 28, 31, 50],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_depth': [-1, 3, 5],
    'n_estimators': [50, 100, 200, 500],
}

# 2. Set up model with grid
LGBM_random = RandomizedSearchCV(estimator = LGBM_base,
                                 param_distributions = random_grid, 
                                 n_iter = 100, cv = 2,
                                 scoring='r2',
                                 verbose= 2,
                                 random_state= 42,
                                 n_jobs = -1)


In [ ]:
# 3. Base LGBM on base data
trainAndGetScore(LGBM_random, "LGBM_random", Bldg_df, scores_df)

# 4. Base LGBM on cyclica time features
trainAndGetScore(LGBM_random, "LGBM_random_cyclic", Bldg_cyclic, scores_df)
scores_df

### 3.6.3 Bayesian Opt. tuning

In [ ]:
# cv_results = lgb.cv(
#         params,
#         dftrainLGB,
#         num_boost_round=100,
#         nfold=3,
#         metrics='mae',
#         early_stopping_rounds=10,
#         stratified=False
#         )